<a href="https://colab.research.google.com/github/hit25082000/PLN-Filmes/blob/main/filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Link para base de dados, que foi adicionada ao google drive:
#https://drive.google.com/file/d/18SdLpPZQUKKqai7qdK-GZY-39-DNnC0a/view?usp=sharing

#Baixar base de dados em google drive por meio do link de acesso de edição via link
! pip install --upgrade gdown
! gdown 18SdLpPZQUKKqai7qdK-GZY-39-DNnC0a -O "archive.zip"
! unzip "archive.zip"

  Attempting uninstall: gdown
    Found existing installation: gdown 5.1.0
    Uninstalling gdown-5.1.0:
      Successfully uninstalled gdown-5.1.0
Downloading...
From (original): https://drive.google.com/uc?id=18SdLpPZQUKKqai7qdK-GZY-39-DNnC0a
From (redirected): https://drive.google.com/uc?id=18SdLpPZQUKKqai7qdK-GZY-39-DNnC0a&confirm=t&uuid=d4d0dbd9-71bc-4d24-8980-097b418f5075
To: /content/archive.zip
100% 50.8M/50.8M [00:00<00:00, 63.0MB/s]
Archive:  archive.zip
  inflating: imdb-reviews-pt-br.csv  


In [ ]:
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from multiprocessing import Pool

In [ ]:
# Baixe os dados necessários do nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import spacy
!python -m spacy download pt_core_news_sm
nlp = spacy.load('pt_core_news_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 63.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
resenha = pd.read_csv("imdb-reviews-pt-br.csv")

In [ ]:
resenha_menor = resenha.sample(5000)

In [ ]:
resenha_menor

,id,text_en,text_pt,sentiment
16332,16334,A truly accurate and unglamourous look into mo...,Um olhar verdadeiramente preciso e sem graça p...,pos
9740,9742,I picked out this DVD out of the cheepo bin at...,Eu escolhi este DVD da caixa de cheepo no Walm...,neg
31624,31626,Nothing like a movie about a group of friends ...,Nada como um filme sobre um grupo de amigos qu...,neg
3757,3759,This has got to be the worst case of over acti...,Este tem que ser o pior caso de atuação excess...,neg
27801,27803,"If you really, truly want to waste two hours o...",Se você realmente quiser realmente desperdiçar...,neg
...,...,...,...,...
4069,4071,"All this show is, is the same plot. Kuszko spe...","Tudo isso é show, é o mesmo enredo. Kuszko ort...",neg
41588,41590,Despite Louis B. Mayer reportedly not being in...,Apesar de Louis B. Mayer supostamente não esta...,pos
20712,20714,This episode of Buffy was one of my personal f...,Este episódio de Buffy foi um dos meus favorit...,pos
22480,22482,When I first saw this film around 6 months ago...,Quando vi pela primeira vez este filme há cerc...,pos


In [ ]:
resenha_menor['sentiment'].value_counts()

sentiment
pos    2532
neg    2468
Name: count, dtype: int64

In [ ]:
#Mostra classes existentes na base de dados
resenha_menor['sentiment'].unique()

array(['pos', 'neg'], dtype=object)

In [ ]:
#Mostra se há algum registro com valor NaN na tabela
resenha_menor['sentiment'].isna().sum()

0

In [ ]:
stopwords.words('portuguese')

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'hou

In [ ]:
#Função de pré-processamento dos textos
def preprocess_text(text):
    if isinstance(text, str):  # Verifique se o texto é uma string
        # Remover caracteres especiais
        text = re.sub(r'[^A-Za-z\s]', '', text)

        # Transformar em minúsculo
        text = text.lower()

        # Tokenizar
        doc = nlp(text)

        # Lematização e remoção de stopwords
        tokens = [token.lemma_ for token in doc if token.lemma_ not in stopwords.words('portuguese')]

        return ' '.join(tokens)
    else:
        return ''  # Retorne uma string vazia se o valor não for uma string

In [ ]:
preprocess_text("O aluno era muito estudioso, estudou por horas e horas e se deu bem na prova!")

'aluno estudioso estudar hora hora dar bem em o prova'

In [ ]:
#Pré-processamento dos textos
with Pool() as pool:
    resenha_menor['text_pt'] = pool.map(preprocess_text, resenha_menor['text_pt'])

In [ ]:
X = resenha_menor['text_pt']
y = resenha_menor['sentiment']

In [ ]:
df_reduzido = pd.DataFrame({'text_pt': X, 'sentiment': y})

In [ ]:
df_reduzido['sentiment'].value_counts()

sentiment
pos    2532
neg    2468
Name: count, dtype: int64

In [ ]:
df_reduzido.iloc[0]['text_pt']

'olhar verdadeiramente preciso graa vida moderno poder definir qualquer cidade de o reino unir morar conjunto habitacional glasgow poder relacionar bem filme infelizmente situaes personagem so realista em o previsvel ator so incrivelmente crvel sentir espionar vizinho mergulho ntimo em o vida pessoa frgel esperana fiquei comover algum cena adorar modo filme filmar geral filme   imperdvel'

In [ ]:
vectorizer = CountVectorizer()
#Ajuste do vetorizador aos textos para se obter o vocabulário e os índices de cada palavra do vocabulário
vectorizer.fit(df_reduzido['text_pt'])

CountVectorizer()

In [ ]:
#Extração de todo o vocabulário da base de dados
vectorizer.vocabulary_

{'olhar': 27616,
 'verdadeiramente': 39637,
 'preciso': 30185,
 'graa': 17548,
 'vida': 39804,
 'moderno': 25577,
 'poder': 29714,
 'definir': 9978,
 'qualquer': 31244,
 'cidade': 6996,
 'de': 9783,
 'reino': 32245,
 'unir': 39188,
 'morar': 25803,
 'conjunto': 8213,
 'habitacional': 18016,
 'glasgow': 17257,
 'relacionar': 32283,
 'bem': 4036,
 'filme': 15631,
 'infelizmente': 20089,
 'situaes': 35385,
 'personagem': 29159,
 'so': 35602,
 'realista': 31766,
 'em': 12708,
 'previsvel': 30423,
 'ator': 2945,
 'incrivelmente': 19908,
 'crvel': 9298,
 'sentir': 34637,
 'espionar': 13976,
 'vizinho': 40099,
 'mergulho': 25031,
 'ntimo': 27163,
 'pessoa': 29251,
 'frgel': 16408,
 'esperana': 13930,
 'fiquei': 15766,
 'comover': 7769,
 'algum': 1107,
 'cena': 6445,
 'adorar': 568,
 'modo': 25587,
 'filmar': 15628,
 'geral': 17038,
 'imperdvel': 19578,
 'escolher': 13703,
 'dvd': 12269,
 'caixa': 5599,
 'cheepo': 6741,
 'walmart': 40331,
 'porque': 29940,
 'capa': 5876,
 'mostrar': 25936,
 'a

In [ ]:
#Gerar representação
representacao = vectorizer.transform(df_reduzido['text_pt'])
representacao

<5000x41270 sparse matrix of type '<class 'numpy.int64'>'
	with 495179 stored elements in Compressed Sparse Row format>

In [ ]:
sorted(vectorizer.vocabulary_.items())

[('2e', 0),
 ('aa', 1),
 ('aaa', 2),
 ('aachen', 3),
 ('aag', 4),
 ('aaghh', 5),
 ('aakash', 6),
 ('aamir', 7),
 ('aankhen', 8),
 ('aardman', 9),
 ('aargh', 10),
 ('aaron', 11),
 ('aaronanchors', 12),
 ('aaronar', 13),
 ('aaronr', 14),
 ('aarons', 15),
 ('aashok', 16),
 ('aasmaan', 17),
 ('aasman', 18),
 ('aawip', 19),
 ('ab', 20),
 ('aba', 21),
 ('abacaxi', 22),
 ('abade', 23),
 ('abadia', 24),
 ('abadre', 25),
 ('abafado', 26),
 ('abafador', 27),
 ('abafar', 28),
 ('abagail', 29),
 ('abaixada', 30),
 ('abaixar', 31),
 ('abaixarse', 32),
 ('abaixeir', 33),
 ('abaixo', 34),
 ('abaixoisso', 35),
 ('abalar', 36),
 ('abanar', 37),
 ('abandonada', 38),
 ('abandonado', 39),
 ('abandonar', 40),
 ('abandonem', 41),
 ('abandonlo', 42),
 ('abandono', 43),
 ('abarrotar', 44),
 ('abastados', 45),
 ('abastecer', 46),
 ('abastecir', 47),
 ('abate', 48),
 ('abater', 49),
 ('abatir', 50),
 ('abba', 51),
 ('abbadr', 52),
 ('abbar', 53),
 ('abbey', 54),
 ('abbora', 55),
 ('abbott', 56),
 ('abbu', 57),


In [ ]:
termos = [item[0] for item in sorted(vectorizer.vocabulary_.items())]

In [ ]:
df_representacao = pd.DataFrame(representacao.toarray(), columns=termos)

In [ ]:
df_representacao

,2e,aa,aaa,aachen,aag,aaghh,aakash,aamir,aankhen,aardman,...,zumbir,zumbis,zumbisr,zuniga,zurer,zvonimir,zwart,zwartr,zwick,zyuranger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Retirar palavras com mais de 30 letras da base de dados e realizar a classificação novamente com regressão logística.
def remove_long_words(text):
    return ' '.join([word for word in text.split() if len(word) <= 30])

In [ ]:
#Classificação com regressão logistica
df_reduzido_no_long_words = df_reduzido['text_pt'].apply(remove_long_words)

vectorizer.fit(df_reduzido_no_long_words)
df_representacao = vectorizer.transform(df_reduzido_no_long_words)

treino, teste, classe_treino, classe_teste = train_test_split(df_representacao,
                                                              df_reduzido['sentiment'],
                                                              random_state = 42)

In [ ]:
#Treinar o modelo de regressão logística
model = LogisticRegression(max_iter=1000)
model.fit(treino, classe_treino)

#Mostar classificação
y_pred = model.predict(teste)
accuracy_score(classe_teste, y_pred)
print("Relatório de Classificação:")
print(classification_report(classe_teste, y_pred))

Relatório de Classificação:
              precision    recall  f1-score   support

         neg       0.84      0.84      0.84       642
         pos       0.84      0.84      0.84       608

    accuracy                           0.84      1250
   macro avg       0.84      0.84      0.84      1250
weighted avg       0.84      0.84      0.84      1250



In [ ]:
#Mostrar classificação sem palavras grandes
vectorizer.fit(df_reduzido_no_long_words)
vectorizer.vocabulary_
representacao = vectorizer.transform(df_reduzido_no_long_words)
termos = [item[0] for item in sorted(vectorizer.vocabulary_.items())]
df_representacao = pd.DataFrame(representacao.toarray(), columns=termos)
df_representacao

,2e,aa,aaa,aachen,aag,aaghh,aakash,aamir,aankhen,aardman,...,zumbir,zumbis,zumbisr,zuniga,zurer,zvonimir,zwart,zwartr,zwick,zyuranger
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
